In [2]:
import nbimporter
import math
import numpy as np
import sys
if('..' not in sys.path):
    sys.path.insert(0,'..')
from Maths.Action import Action
from Maths.State import State
from Maths.Cord import Cord
from Maze.Maze import Maze
from Agents.Prey import Prey
import random
from Maths.DQNSolver import DQNSolver
from Maths.Action import Action

class Worker(Prey):
    sid=0
    def __init__(self, maze,span):
        super().__init__("Worker",Worker.sid)  
        Worker.sid+=1
        self.S=State.ENTRANCE
        self.m=maze
        self.Pos=Cord(maze.getInitialX(),maze.getInitialY())
        self.exploring=True
        self.savedState=[]
        self.savedTime=None
        self.queenPos=self.Pos
        self.routeToQueen=[]
        self.vulnerable=False
        
        action_space=[]
        
        for i in range(0,len(Action)):            
            action_space.append(i)
            
        self.action_space_worker=np.asarray(action_space)
        self.observation_space_worker= math.pow(2*span+1,2)
        
        self.dqn_solver_worker = DQNSolver(int(self.observation_space_worker), len(self.action_space_worker))
        #self.RandomWalk()
        
    def updateVulnerability(self,pList,queenPos):
        north=Cord(self.Pos.X,self.Pos.Y+1)
        south=Cord(self.Pos.X,self.Pos.Y-1)
        east=Cord(self.Pos.X+1,self.Pos.Y)
        west=Cord(self.Pos.X-1,self.Pos.Y)
        
        nextToQueen=False
        if((queenPos.equals(north)) or (queenPos.equals(south)) or (queenPos.equals(east)) or (queenPos.equals(west))):
            nextToQueen=True
        
        if((north in pList) or (south in pList) or (east in pList) or (west in pList) or (nextToQueen)):
            self.vulnerable=False
        else:
            self.vulnerable=True
        
    def isQueenOnEdge(self,span,pList,qList,mList):
        if(not self.exploring):
            return False
        view=self.getAugView(self.Pos,span,pList,qList,mList)
        edge=False
        for cell in view[0]:
            if cell==7:
                edge=True
        for cell in view[len(view)-1]:
            if cell==7:
                edge=True
        for row in view:
            if row[0]==7:
                edge=True
            if row[len(row)-1]==7:
                edge=True
        if(edge):
            #self.save_state(span,pList,qList,mList)
            self.exploring=False
            return True
        else:
            return False
        
    def save_state(self,span,pList,qList,mList):
        #print("SNAPPING")
        self.savedState=self.getAugView(self.Pos,span,pList,qList,mList)
        self.savedTime=self.getTime()
        #print("endSnap")
        
    def goToQueen(self,queenPos,span,pList,qList,mList,wReward_not_possible, wReward_wall, wReward_toQueen, wReward_atQueen, wReward_repeat_pos, wReward_else):
        #print("3")
        """state=self.getAugView(self.Pos,span,pList,qList,mList)
        state = np.reshape(state,  [1,int(self.observation_space_worker)])
        #print("4")
        ownPos=self.Pos #Cord(span,span)    
        for i in range(len(state)):
            for j in range(len(state[i])):
                if state[i][j]==7:
                    self.queenPos=Cord(i+self.Pos.X-span,j+self.Pos.Y-span)
        #print(ownPos.CordToString(), queenPos.CordToString())
        #print("5")
        action = self.dqn_solver_worker.act(state)
        #print("6")
        state_next, reward, terminal = self.step(action,span,pList,qList,mList,wReward_not_possible, wReward_wall, wReward_toQueen, wReward_atQueen, wReward_repeat_pos, wReward_else)
        #print("7")
        state_next = np.reshape(state_next, [1,int(self.observation_space_worker)])
        #print("8")
        self.dqn_solver_worker.remember(state, action, reward, state_next, terminal)
        #print("9",state,state_next)
        self.dqn_solver_worker.experience_replay()
        #print("10")"""
        
        self.routeToQueen=self.m.GetRoute(self.Pos,queenPos)
        oldPosition=self.Pos
        next_step=self.Pos
        if(len(self.routeToQueen)>1):
            next_step=self.routeToQueen[1]
        action=4
        if(next_step.Y==self.Pos.Y+1):
            action=0
        elif(next_step.Y==self.Pos.Y-1):
            action=1
        elif(next_step.X==self.Pos.X+1):
            action=2
        elif(next_step.X==self.Pos.X-1):
            action=3
        if(action in self.m.WhichWayIsClear(self.Pos, True)):
            self.Do(action,self.m)
            reward=self.getRewardToQueen(self.getPos(),True,oldPosition,self.getAugView(self.getPos(),span,pList,qList,mList), wReward_not_possible, wReward_wall, wReward_toQueen, wReward_atQueen, wReward_repeat_pos, wReward_else)
              
        
        else:      
            self.Do(4,self.m)
            reward=self.getRewardToQueen(self.getPos(),False,oldPosition,self.getAugView(self.getPos(),span,pList,qList,mList), wReward_not_possible, wReward_wall, wReward_toQueen, wReward_atQueen, wReward_repeat_pos, wReward_else)
       
        state_next=self.getAugView(self.Pos,span,pList,qList,mList)
        return state_next,reward
        
        #take step
    def step(self,action,span,pList,qList,mList,wReward_not_possible, wReward_wall, wReward_toQueen, wReward_atQueen, wReward_repeat_pos, wReward_else):
        reward=0
        terminal=False
        info={}
        
        oldPosition=self.getPos()
        state_Next=np.empty(1)      
        queenPos=self.queenPos
        #print("a")
                  
        if(self.action_space_worker[action] in self.m.WhichWayIsClear(oldPosition, True)):
            self.Do(self.action_space_worker[action],self.m)
            #print("b")
            state_Next=np.asarray(self.getAugView(self.getPos(),span,pList,qList,mList))         
            #print("c")
            #self, pos, queenPos, possible, oldPos, view, rNotPos, rWall, rToQ, rAtQ, rRep, rElse
            reward+=self.getRewardToQueen(self.getPos(), True,oldPosition,self.getAugView(self.getPos(),span,pList,qList,mList), wReward_not_possible, wReward_wall, wReward_toQueen, wReward_atQueen, wReward_repeat_pos, wReward_else)
            
        else:              
            self.Do(self.action_space_worker[4],self.m)
            #print("d")
            state_Next=np.asarray(self.getAugView(self.getPos(),span,pList,qList,mList))             
            #print("e")
            reward+=self.getRewardToQueen(self.getPos(),False,oldPosition,self.getAugView(self.getPos(),span,pList,qList,mList), wReward_not_possible, wReward_wall, wReward_toQueen, wReward_atQueen, wReward_repeat_pos, wReward_else)
                  
        view=self.getAugView(self.getPos(),span,pList,qList,mList)    
        if(view[span][span]==7):
            self.exploring=True
            terminal=True
        #print(state_Next,reward,terminal)
        return state_Next, reward, terminal
        
    def NextMove(self):
        direction=self.RandomWalk()        
        self.Do(direction,self.m)
        
    def RandomWalk(self):
        now=self.getPos()
        possibleDirections=self.m.WhichWayIsClear(now)
        direction=possibleDirections[random.randint(0,len(possibleDirections)-1)]
        return direction
        
    def Do(self, direction, m):
        if(direction==0):
            self.MoveNorth(m)
        if(direction==1):
            self.MoveSouth(m)
        if(direction==2):
            self.MoveEast(m)
        if(direction==3):
            self.MoveWest(m)
        if(direction==4):
            self.StayStill(m)
        newC=self.getPos()
        if(newC in m.getExit()):
            self.S=State.EXIT
        else:
            self.S=State.CLEAR
            
    def getSetView(self,maze, span):
        views=[]
        for i in range(0,maze.getHeight()):
            for j in range(0, maze.getWidth()):
                c= Cord(i,j)
                view=self.getView(c,span)
                if(view not in views):
                    views.append(view)
        return views
    
    def getView(self,c, span):
        #view=[[]for i in range(2*span+1)]
        view=[]
        for i in range(c.X-span,c.X+span+1):
            row=[]
            for j in range(c.Y-span,c.Y+span+1):
                row.append(self.m.getMazePart(i,j))
            #view[i-c.X-span]=row
            view.append(row)
        return view
    
    def getAugView(self,c, span, wPos, qPos, mPos):
        #view=[[]for i in range(2*span+1)]
        #print("6")
        view=[]
        for i in range(c.X-span,c.X+span+1):
            row=[]
            #print("7")
            for j in range(c.Y-span,c.Y+span+1):
                val=self.m.getMazePart(i,j)
                #print("8",[w.CordToString for w in wPos],[w.CordToString for w in qPos],[w.CordToString for w in mPos])
                if(Cord(i,j) in wPos):
                    val=4
                #print("8.1")
                if(Cord(i,j) in qPos):
                    val=7
                #print("8.2")
                if(Cord(i,j) in mPos):
                    val=6    
                #print("8.3")
                row.append(val)
                #print("8.4",val)
            #view[i-c.X-span]=row
            #print("9")
            view.append(row)
        #print("10")
        return view
    
    def getRewardToQueen(self, pos, possible, oldPos, view, rNotPos, rWall, rToQ, rAtQ, rRep, rElse):
        queenPos=self.queenPos
        #print("rewrd part",pos,oldPos,queenPos)
        if not(possible):
            return rNotPos
        else:
            if(self.m.CheckWall(pos)):
                return rWall
            if(self.m.CheckClear(pos)):
                for row in view:
                    if (7 in row):
                        if(queenPos.equals(pos)):
                            return rAtQ
                        if((math.pow(queenPos.X-pos.X,2)+math.pow(queenPos.Y-pos.Y,2))<(math.pow(queenPos.X-oldPos.X,2)+math.pow(queenPos.Y-oldPos.Y,2))):
                            return rToQ
                if (oldPos.equals(pos)):
                    return rRep
                else:
                    return rElse
  
    def getReward(self, pos, possible, oldPos, view, rNotPos, rWall, rEnt, rFinBef, rEx, rToEx, rRep, rElse):
        
        if not(possible):
            return rNotPos
        else:
            if(self.m.CheckWall(pos)):
                return rWall
            if(self.m.CheckEntrance(pos)):
                return rEnt
            if (oldPos.equals(pos) and self.m.CheckExit(pos)):
                    return rFinBef
            if(self.m.CheckExit(pos)):
                return rEx
            if(self.m.CheckClear(pos)):
                exits=self.m.Exits
                for row in view:
                    if (3 in row):
                        for exit in exits:
                            if((math.pow(exit.X-pos.X,2)+math.pow(exit.Y-pos.Y,2))<(math.pow(exit.X-oldPos.X,2)+math.pow(exit.Y-oldPos.Y,2))):
                                return rToEx
                if (oldPos.equals(pos)):
                    return rRep
                else:
                    return rElse

Importing Jupyter notebook from ../Maths/Action.ipynb
Importing Jupyter notebook from ../Maths/State.ipynb
Importing Jupyter notebook from ../Maths/Cord.ipynb
Importing Jupyter notebook from ../Maze/Maze.ipynb
Importing Jupyter notebook from ../Agents/Prey.ipynb
Importing Jupyter notebook from ../Agents/Agent.ipynb
Importing Jupyter notebook from ../Maths/DQNSolver.ipynb


Using TensorFlow backend.


In [4]:
def show_span(span):
    for row in span:
        print(row)
    print(" ")
m1= Maze("Test,10,10,4,0,4,9,1111211111100000000111100110011000000001100001011111011000111000000001100101010110000000011111311111")

w1=Worker(m1,6)
c=Cord(4,0)
v=w1.getAugView(c,6,[Cord(4,1)],[Cord(4,6)],[Cord(4,3)])
#print("View:",v)
print(" ")
#print(m1.getMazePart(4,0))
show_span(v)
w1.isQueenOnEdge(6,[Cord(4,1)],[Cord(4,6)],[Cord(4,3)])
#print(w1.savedState, w1.savedTime)
w1.goToQueen(6,[Cord(4,1)],[Cord(4,6)],[Cord(4,3)])
#print(m1.getInitialX(), m1.getInitialY())

 
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0]
[1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0]
[1, 1, 1, 1, 1, 1, 2, 4, 0, 6, 0, 1, 7]
[1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 


TypeError: goToQueen() missing 7 required positional arguments: 'mList', 'wReward_not_possible', 'wReward_wall', 'wReward_toQueen', 'wReward_atQueen', 'wReward_repeat_pos', and 'wReward_else'

In [7]:
from Maths.Action import Action
print(Action(0))

Action.NORTH
